# Cricket Scores GUI
> This app provides a GUI for checking cricket scores using objects

- toc:true
- branch: master
- badges: true
- comments: true
- author: Rohan Juneja
- categories: [pbl, cb]

In [2]:
import java.util.List;

In [3]:
%%loadFromPOM
<dependency>
    <groupId>org.json</groupId>
    <artifactId>json</artifactId>
    <version>20090211</version>
</dependency>

The ``CricketScores`` class uses static methods in order to pull cricket scores for a specific team

In [4]:
import java.net.http.*;
import org.json.*;
import java.util.List;

public class CricketScores {
  private static Map<String,String> teamIds = new HashMap<>();
  static {
    teamIds.put("IND", "187765");
    teamIds.put("AFG", "187575");
    teamIds.put("SL", "187756");
    teamIds.put("PAK", "187754");
  }
  public static List<String[]> getScores (String team) throws IOException, InterruptedException, JSONException {
    String teamId = teamIds.get(team);
    HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://cricketapi10.p.rapidapi.com/api/cricket/team/"+teamId+"/matches/previous/0"))
		.header("X-RapidAPI-Key", "f58766a4c1msh85336d65179f2c3p15dc2bjsne94dee50a809")
		.header("X-RapidAPI-Host", "cricketapi10.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
    HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
    JSONArray events = (new JSONObject(response.body())).getJSONArray("events");
    List<String[]> ret = new ArrayList<String[]>();
    for (int i = events.length()-1; i>Math.max(0, events.length()-5); i--) {
      JSONObject event = (JSONObject) events.get(i);
      String awayName = event.getJSONObject("awayTeam").getString("name");
      int awayScore = event.getJSONObject("awayScore").getJSONObject("innings").getJSONObject("inning1").getInt("score");
      int awayWickets = event.getJSONObject("awayScore").getJSONObject("innings").getJSONObject("inning1").getInt("wickets");
      int awayOvers = event.getJSONObject("awayScore").getJSONObject("innings").getJSONObject("inning1").getInt("overs");
      String homeName = event.getJSONObject("homeTeam").getString("name");
      int homeScore = event.getJSONObject("homeScore").getJSONObject("innings").getJSONObject("inning1").getInt("score");
      int homeWickets = event.getJSONObject("homeScore").getJSONObject("innings").getJSONObject("inning1").getInt("wickets");
      int homeOvers = event.getJSONObject("homeScore").getJSONObject("innings").getJSONObject("inning1").getInt("overs");
      String[] data = {event.getString("note"), awayName + ": " + awayScore + "/" + awayWickets + " in " + awayOvers + " overs", homeName + ": " + homeScore + "/" + homeWickets + " in " + awayOvers + " overs"};
      ret.add(data);
    }
    return ret;
  }
}

Here, the CricketJFrame is a class which can be instantiated as an object in order to create the Window which provides Cricket Scores for 3 teams (IND, SL, PAK) which can be chosen from the Menu items (Note that there may be an error for kernel crashing as this occurs once the window is closed)

In [5]:
import java.awt.*;
import java.awt.event.*;
import javax.swing.*;
import java.util.Timer;
import java.util.TimerTask;
// Graphical-User-Interface for Desktop in Java using Java Swing. 
public class CricketJFrame extends JFrame implements ActionListener {
    private JFrame frame;
    private JMenuBar menubar;
    private JMenu menu;
    private JLabel message = new JLabel("Click on Team to select a team.");
    private JLabel[][] matches = {
      {new JLabel(""), new JLabel("")},
      {new JLabel(""), new JLabel("")},
      {new JLabel(""), new JLabel("")},
      {new JLabel(""), new JLabel("")}
    };
    public final String[] MENUS = { // 1D Array of Menu Choices
        "IND", "SL", "PAK"  
    };
    JPanel labelPanel = new JPanel();

    // Constructor enables the Frame instance, the object "this.frame"
    public CricketJFrame() {
	      // Initializing Key Objects
        frame = new JFrame("Cricket Scores");
	      menubar = new JMenuBar();
	      menu = new JMenu("Teams");

        // Initialize panel & message
        labelPanel.setLayout(new BoxLayout(labelPanel, BoxLayout.Y_AXIS));
        message.setAlignmentX(Component.CENTER_ALIGNMENT);
        message.setVisible(false);

        // Initializing Menu objects and adding actions
        for (String mx : MENUS) {
            JMenuItem m = new JMenuItem(mx);
            m.addActionListener(this);
            menu.add(m); 
        }

        // Adding / Connecting Objects
        menubar.add(menu);
        frame.setJMenuBar(menubar);
        labelPanel.add(message);
        labelPanel.add(Box.createGlue());

        //Adding match JLabels
        for (JLabel[] lab : matches) {
          lab[0].setAlignmentX(Component.CENTER_ALIGNMENT);
          lab[1].setAlignmentX(Component.CENTER_ALIGNMENT);
          lab[0].setVisible(false);
          lab[1].setVisible(false);
          labelPanel.add(lab[0]);
          labelPanel.add(lab[1]);
          labelPanel.add(Box.createGlue());
        }

        frame.add(labelPanel);

        // Sets JFrame close operation to Class variable JFrame.EXIT_ON_CLOSE
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        
        // set the size of window based on objects
        frame.setSize(400,400);

        // makes the frame object visible according to properties previously set
        frame.setVisible(true); 
    }

    // event from user selecting a menu option
    public void actionPerformed(ActionEvent e) {
        String selection = e.getActionCommand();  // menu selection
        String msg; // local variable to create response from action

        // run code based on the menuItem that was selected
        try {
            List<String[]> teamData = CricketScores.getScores(selection);
            message.setText(selection + " Recent Cricket Matches");
            message.setVisible(true);
            for (int i = 0; i<teamData.size(); i++) {
              System.out.println(i);
              // matches[i].setText("<html><div style='text-align: center; background: gray;'>" + teamData.get(i)[0] + "<br/>" + teamData.get(i)[1] + "<br/>" + teamData.get(i)[2] + "</div></html>");
              // matches[i].setVisible(true);
              matches[i][0].setText(teamData.get(i)[0]);
              matches[i][1].setText(teamData.get(i)[1] + "      " + teamData.get(i)[2]);
              matches[i][0].setVisible(true);
              matches[i][1].setVisible(true);
            }
        } catch (Exception ex) {
          System.err.println("Error occured");
        }
    }

    // Driver turn over control the GUI
    public static void main(String[] args) {
        // Activates an instance of CricketJFrame class, which makes a JFrame object
        new CricketJFrame();
    }
}
CricketJFrame.main(null);

: 